# **Hands-on Lab: Build an Interactive Dashboard with Ploty Dash**
In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

This dashboard application contains input components such as a dropdown list and a range slider to
interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component

TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

TASK 3: Add a Range Slider to Select Payload

TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

***Note:Please take screenshots of the Dashboard and save them. Further upload your notebook to github.***

***The github url and the screenshots are later required in the presentation slides.***

Your completed dashboard application should look like the following screenshot:



In [4]:
# Import required libraries
import pandas as pd
# !pip install dash
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

success_launch = spacex_df.groupby('Launch Site')['class'].mean().reset_index()
print(success_launch)

# Create an interactive pie chart using the px.pie():
fig = px.pie(success_launch, names= 'Launch Site', values='class', title="Total Success Launches By Site")
fig.update_traces(textinfo="label+percent", textposition="inside", textfont_size=14)
fig.show()


    Launch Site     class
0   CCAFS LC-40  0.269231
1  CCAFS SLC-40  0.428571
2    KSC LC-39A  0.769231
3   VAFB SLC-4E  0.400000


In [5]:
# Create the interactive scatter plot:
px.scatter(spacex_df, x="Payload Mass (kg)", y="class", color="Launch Site", title="Correlation between Payload and Success for all Sites")


In [6]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(), # Inserts a blank line
                                dcc.Dropdown(id='site-dropdown',
                                options=[{'label': 'All Sites', 'value': 'ALL'},
                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                         ],
                                value='All',
                                placeholder= "Select a Launch Site here",
                                searchable=True
                                ),
                                # TASK 2: Add a dcc.Graph for success_pie_chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success_pie_chart')),
                                html.Br(),

                                # TASK 3: Add a dcc.Graph for success_payload_scatter_chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success_payload_scatter_chart')),

                                # TASK 4: Add a slider to select payload range for success_payload_scatter_chart
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='id', min=0, max=10000, step=1000, marks={0: '0', 100: '100'}, value=[min_payload, max_payload]),

                                ])

# TASK 5: Add a callback function to render success_pie_chart based on selected site dropdown:
# Add a callback function for `site-dropdown` as input, `success_pie_chart` as output
@app.callback(Output(component_id='success_pie_chart', component_property='figure'), Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Group by Launch Site and sum success (class=1)
        df = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        fig = px.pie(df, values='class', names='Launch Site',
                     title='Total Success Launches by Site')
    else:
        # Filter by specific site and count success vs failure
        df = spacex_df[spacex_df['Launch Site'] == entered_site]
        outcome_counts = df['class'].value_counts().reset_index()
        outcome_counts.columns = ['class', 'count']
        fig = px.pie(outcome_counts, values='count', names='class', title=f'Total Success Launches for Site {entered_site}')
    return fig

# TASK 6: Add a callback function to render the success_payload_scatter_chart scatter plot:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success_payload_scatter_chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='id', component_property='value')]
)

def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= low) &
        (spacex_df['Payload Mass (kg)'] <= high)
    ]

    fig = px.scatter(filtered_df,
                     x='Payload Mass (kg)',
                     y='class',
                     color='Booster Version Category',
                     title='Correlation between Payload and Success for all Sites')

    return fig

# Run the app
if __name__ == '__main__':
    app.run()


<IPython.core.display.Javascript object>